# Numpy tutorial

Numpy's speedup is no joke.

**Normal Python:**

In [2]:
import random
data = []
for i in range(1000000):
    data.append(random.gauss(0, 1))

In [3]:
%%timeit
data2 = []
for x in data:
    data2.append(x**2)

1 loop, best of 3: 513 ms per loop


**Numpy:**

In [4]:
import numpy
data = numpy.random.normal(0, 1, 1000000)

In [5]:
%%timeit
data2 = data**2

100 loops, best of 3: 3.93 ms per loop


**A Numpy array is everything normal Python data is not:**

   * Loop performed in native bytecode
   * Type-checking performed once before loop
   * Data are packed in contiguous bytes
   * Python's Global Interpreter Lock (GIL) is released during loop

**Bonus:**

   * Most methods benefit from hardware vectorization

But you have to write your algorithms "sideways."

In [6]:
px = numpy.random.normal(0, 30, 100000)
py = numpy.random.normal(0, 30, 100000)
pz = numpy.random.normal(0, 300, 100000)

**Computing one event at a time:**

In [7]:
%%timeit
p = numpy.empty(100000)
for i in range(len(p)):                                   # for each px[i], py[i], pz[i]
    p[i] = numpy.sqrt(px[i]**2 + py[i]**2 + pz[i]**2)     # compute p[i]

1 loop, best of 3: 1.52 s per loop


**Computing one column at a time:**

In [8]:
%%timeit
p = numpy.sqrt(px**2 + py**2 + pz**2)       # compute all px**2, then all py**2, then all pz**2, then sum all, then sqrt all

100 loops, best of 3: 3.94 ms per loop


Normal math functions are *scalar* (e.g. binary operators like `+` or functions from `import math`). They perform one operation per appearance in Python source code.

Numpy math functions are *vectorized.* Given equal-length arrays as input, they return the same length array as output, performing all loops in compiled C code, possibly doing 4 or 8 at a time in the processor.

In [9]:
small_array = numpy.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [10]:
small_array**2

array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81])

In [11]:
numpy.sqrt(small_array)

array([0.        , 1.        , 1.41421356, 1.73205081, 2.        ,
       2.23606798, 2.44948974, 2.64575131, 2.82842712, 3.        ])

In [12]:
# this won't work because math.sqrt wants a scalar number
import math
math.sqrt(small_array)

TypeError: only size-1 arrays can be converted to Python scalars

Numpy arrays are contiguous blocks of bytes in memory, just like C arrays.

In [33]:
small_array.view(numpy.uint8)        # view the 64-bit integers as unsigned 8-bit integers

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0,
       0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 8, 0,
       0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

They can be multidimensional.

In [34]:
twod = small_array.reshape(2, 5)     # view as 2 arrays of 5 elements each
twod

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

They can even be arrays of _structs_.

In [66]:
table = numpy.array([(0, -9, 0.0), (1, -7, 1.1), (2, -5, 2.2), (3, -3, 3.3), (4, -1, 4.4), (5, 1, 5.5), (6, 3, 6.6), (7, 5, 7.7), (8, 7, 8.8), (9, 9, 9.9)], dtype=[("one", numpy.uint8), ("two", numpy.int64), ("three", numpy.double)])
table

array([(0, -9, 0. ), (1, -7, 1.1), (2, -5, 2.2), (3, -3, 3.3),
       (4, -1, 4.4), (5,  1, 5.5), (6,  3, 6.6), (7,  5, 7.7),
       (8,  7, 8.8), (9,  9, 9.9)],
      dtype=[('one', 'u1'), ('two', '<i8'), ('three', '<f8')])

Besides being a common, agreed-upon language for sharing arrays with C/C++ and Fortran code, Numpy has powerful indexing behaviors:

In [29]:
small_array[-1:1:-2]

array([9, 7, 5, 3])

In [28]:
twod[1, ::2]

array([5, 7, 9])

In [40]:
table["three"][5:]

array([5.5, 6.6, 7.7, 8.8, 9.9])

In [45]:
table["three"][[False, False, False, False, False, True, False, True, False, True]]

array([5.5, 7.7, 9.9])

In [46]:
table["three"][[7, 5, 5, 3, 2, 7]]

array([7.7, 5.5, 5.5, 3.3, 2.2, 7.7])

In [57]:
table["three"][table["two"]]

array([1.1, 3.3, 5.5, 7.7, 9.9, 1.1, 3.3, 5.5, 7.7, 9.9])

In [61]:
table["three"][numpy.tile([7, 2], 5)]

array([7.7, 2.2, 7.7, 2.2, 7.7, 2.2, 7.7, 2.2, 7.7, 2.2])

The same rules apply to *assigning* to arrays.

In [67]:
small_array[[False, False, False, False, False, True, False, True, False, True]] = 5000, 7000, 9000

In [68]:
small_array

array([   0,    1,    2,    3,    4, 5000,    6, 7000,    8, 9000])

**Exercise:**

Suppose you're given a zillion `(px, py, pz, E)` 4-vectors and you want `(E, px, py, pz)` 4-vectors. Do it *fast!*

In [71]:
ZILLION = 1000000
fourvectors = numpy.empty((ZILLION, 4))
fourvectors[:, 0] = numpy.random.normal(0, 1, ZILLION)
fourvectors[:, 1] = numpy.random.normal(0, 1, ZILLION)
fourvectors[:, 2] = numpy.random.normal(0, 10, ZILLION)
fourvectors[:, 3] = numpy.random.normal(0, 10, ZILLION)**2
fourvectors[0]

array([ -1.19166285,  -2.10246838,  -5.33774567, 275.73510149])

In [76]:
%%timeit
fourvectors = ???

SyntaxError: invalid syntax (<unknown>, line 1)